In [11]:
import pandas as pd
import math
from string import punctuation
from datasets import load_dataset_builder
from datasets import load_dataset

from functools import reduce



In [2]:
ds_builder = load_dataset_builder("rotten_tomatoes")

In [3]:
print(ds_builder.info.description)
ds_builder.info.features

Movie Review Dataset.
This is a dataset of containing 5,331 positive and 5,331 negative processed
sentences from Rotten Tomatoes movie reviews. This data was first used in Bo
Pang and Lillian Lee, ``Seeing stars: Exploiting class relationships for
sentiment categorization with respect to rating scales.'', Proceedings of the
ACL, 2005.



{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['neg', 'pos'], id=None)}

In [4]:
dataset = load_dataset("rotten_tomatoes")

Found cached dataset rotten_tomatoes (/Users/noway/.cache/huggingface/datasets/rotten_tomatoes/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46)


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
pd.Series(dataset["validation"]["label"]).value_counts()

1    533
0    533
dtype: int64

In [6]:
text_data = pd.Series(dataset["train"]["text"])


text_data = text_data.str.lower()
display(text_data[0])
text_data = text_data.str.replace("[" + punctuation + "]( |$)", "", regex=True)
display(text_data[0])

data = pd.concat([text_data, pd.Series(dataset["train"]["label"])], axis=1)
data.columns=["text", "label"]
data

'the rock is destined to be the 21st century\'s new " conan " and that he\'s going to make a splash even greater than arnold schwarzenegger , jean-claud van damme or steven segal .'

"the rock is destined to be the 21st century's new conan and that he's going to make a splash even greater than arnold schwarzenegger jean-claud van damme or steven segal "

,text,label
0,the rock is destined to be the 21st century's ...,1
1,the gorgeously elaborate continuation of the l...,1
2,effective but too-tepid biopic,1
3,if you sometimes like to go to the movies to h...,1
4,emerges as something rare an issue movie that'...,1
...,...,...
8525,any enjoyment will be hinge from a personal th...,0
8526,if legendary shlockmeister ed wood had ever ma...,0
8527,hardly a nuanced portrait of a young woman's b...,0
8528,interminably bleak to say nothing of boring,0


In [12]:
def find_vocabulary(text_data):
    return reduce(lambda a,b: a | b, text_data.apply(lambda t: set(t.split(" "))))

find_vocabulary(data["text"])

{'',
 "lead's",
 'distancing',
 'cup',
 'studiously',
 'top-notch',
 "'truthabout",
 'merits',
 'frequent',
 'sorts',
 'non-disney',
 'off-putting',
 'double',
 'razzle-dazzle',
 'tripe',
 'sense',
 'distanced',
 'disclosure',
 'roisterous',
 'grotesque',
 'inexorably',
 'apallingly',
 'victor',
 'ambiguous',
 'well-worn',
 'delightfully',
 'yet',
 'welcomes',
 "nesbitt's",
 'gowns',
 'enjoyed',
 'thing',
 'fictionalize',
 'insipid',
 'popcorn',
 'no',
 'aberration',
 '-dull',
 'off',
 'ver',
 'burlap',
 'rodriguez',
 'situation',
 'teenagers',
 'produce',
 'inquisitiveness',
 'pollute',
 'unapologetically',
 'wattage',
 'van',
 'self-hating',
 'forrest',
 'sleeper',
 'specious',
 'predecesora',
 '7',
 'capitalize',
 'nudity',
 'punches',
 'lighting',
 'governs',
 'surgical',
 'divertida',
 'upends',
 'dulls',
 "'we're",
 'quasi-shakespearean',
 'analyze',
 'commercial',
 'overproduced',
 'stars',
 'one-of-a-kind',
 'dramedy',
 'insufferable',
 'combined',
 'disorienting',
 'impart',
 

In [7]:
def train(data, classes):
    V = find_vocabulary(data["text"])
    
    logprior = {}
    n_data = len(data)
    for c in classes:
        n_c = data["label"].value_counts()[c]
        logprior[c] = math.log(n_c/n_data)
        
        
        
        
train(data, [0, 1])